In [1]:
from minio import Minio
from minio.error import S3Error
import os
import subprocess

# --- 1️⃣ Conectar a MinIO ---
client = Minio(
    "minio-host:9000",  # Cambia por tu host
    access_key="admin",
    secret_key="password123",
    secure=False
)

bucket_name = "pfg"
local_folder = "imagenes"
os.makedirs(local_folder, exist_ok=True)

# --- 2️⃣ Listar objetos ---
objects = client.list_objects(bucket_name)

for obj in objects:
    if not obj.object_name.lower().endswith((".jpg", ".jpeg", ".png")):
        continue  # ignorar otros archivos

    local_path = os.path.join(local_folder, os.path.basename(obj.object_name))

    # --- 3️⃣ Descargar imagen ---
    client.fget_object(bucket_name, obj.object_name, local_path)

    # --- 4️⃣ Intentar procesar ---
    try:
        subprocess.run([
            "python", "-m", "main.main",
            "--image_path", local_path,
            "--output_dir", "output/processed_images",
            "--model_type", "PM25"
        ], check=True)

    except subprocess.CalledProcessError:
        # --- 5️⃣ Borrar de MinIO si da error ---
        print(f"Error procesando {obj.object_name}, borrando de MinIO...")
        client.remove_object(bucket_name, obj.object_name)

    finally:
        # borrar archivo local temporal
        if os.path.exists(local_path):
            os.remove(local_path)


MaxRetryError: HTTPConnectionPool(host='minio-host', port=9000): Max retries exceeded with url: /pfg?location= (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001E1588C2ED0>: Failed to resolve 'minio-host' ([Errno 11001] getaddrinfo failed)"))